<a href="https://colab.research.google.com/github/RoksanaMaya/BreastCancer_Prediction/blob/main/BreastCancer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing

In [40]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [41]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Breast Cancer/preprocessed.csv', index_col=0)

In [42]:
data.sample(10)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
317,1,18.22,18.87,118.70,1027.0,0.09746,0.11170,0.11300,0.07950,0.1807,...,21.84,25.00,140.90,1485.0,0.1434,0.27630,0.38530,0.17760,0.2812,0.08198
342,0,11.06,14.96,71.49,373.9,0.10330,0.09097,0.05397,0.03341,0.1776,...,11.92,19.90,79.76,440.0,0.1418,0.22100,0.22990,0.10750,0.3301,0.09080
102,0,12.18,20.52,77.22,458.7,0.08013,0.04038,0.02383,0.01770,0.1739,...,13.34,32.84,84.58,547.8,0.1123,0.08862,0.11450,0.07431,0.2694,0.06878
527,0,12.34,12.27,78.94,468.5,0.09003,0.06307,0.02958,0.02647,0.1689,...,13.61,19.27,87.22,564.9,0.1292,0.20740,0.17910,0.10700,0.3110,0.07592
81,0,13.34,15.86,86.49,520.0,0.10780,0.15350,0.11690,0.06987,0.1942,...,15.53,23.19,96.66,614.9,0.1536,0.47910,0.48580,0.17080,0.3527,0.10160
353,1,15.08,25.74,98.00,716.6,0.10240,0.09769,0.12350,0.06553,0.1647,...,18.51,33.22,121.20,1050.0,0.1660,0.23560,0.40290,0.15260,0.2654,0.09438
432,1,20.18,19.54,133.80,1250.0,0.11330,0.14890,0.21330,0.12590,0.1724,...,22.03,25.07,146.00,1479.0,0.1665,0.29420,0.53080,0.21730,0.3032,0.08075
60,0,10.17,14.88,64.55,311.9,0.11340,0.08061,0.01084,0.01290,0.2743,...,11.02,17.45,69.86,368.6,0.1275,0.09866,0.02168,0.02579,0.3557,0.08020
495,0,14.87,20.21,96.12,680.9,0.09587,0.08345,0.06824,0.04951,0.1487,...,16.01,28.48,103.90,783.6,0.1216,0.13880,0.17000,0.10170,0.2369,0.06599
292,0,12.95,16.02,83.14,513.7,0.10050,0.07943,0.06155,0.03370,0.1730,...,13.74,19.93,88.81,585.4,0.1483,0.20680,0.22410,0.10560,0.3380,0.09584


In [43]:
# Separate features (X) and target variable (y)
X = data.drop(columns=['diagnosis'])
y = data['diagnosis']

In [44]:
# Initialize models with default hyperparameters
models = {
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': XGBClassifier()
}

In [45]:
# Initialize stratified k-fold cross-validation
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Define evaluation metrics
scoring = ['accuracy', 'precision', 'recall', 'f1']

In [46]:
# Dictionary to store evaluation scores
evaluation_scores = {'Model': []}
for metric in scoring:
    evaluation_scores[metric] = []

# Fit models and evaluate performance
for model_name, model in models.items():
    best_score = 0  # Initialize best score
    best_fold = None  # Initialize best fold

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # Check if the current fold has the best score
        if accuracy > best_score:
            best_score = accuracy
            best_fold = [accuracy, precision, recall, f1]

    # Store the model name and evaluation scores
    evaluation_scores['Model'].append(model_name)
    for i, metric in enumerate(scoring):
        evaluation_scores[metric].append(best_fold[i])



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [47]:
# Convert evaluation scores dictionary to DataFrame
metrics_df = pd.DataFrame(evaluation_scores)

In [48]:
# Print the metrics DataFrame
metrics_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Breast Cancer/evaluation_metrics.csv')
print(metrics_df)

                 Model  accuracy  precision    recall        f1
0        Random Forest  0.964912   0.933333  0.976744  0.954545
1                  SVM  0.938596   0.950000  0.883721  0.915663
2                  KNN  0.956140   0.911111  0.976190  0.942529
3  Logistic Regression  0.973451   1.000000  0.928571  0.962963
4          Naive Bayes  0.964912   0.953488  0.953488  0.953488
5    Gradient Boosting  0.973451   0.975610  0.952381  0.963855
6              XGBoost  0.973684   0.934783  1.000000  0.966292
